In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.image as mpimg
from datetime import datetime
import matplotlib.pyplot as plt

### Let's start by processing the colored cats & dog data 

In [2]:
cats_3 = np.load('cats_dogs_64-128/cats_1000_128_128_3.npy')

In [3]:
dogs_3 = np.load('cats_dogs_64-128/dogs_1000_128_128_3.npy')

In [4]:
cats_3.shape # investigate the shape

(1000, 128, 128, 3)

In [5]:
dogs_3.shape # investigate the shape

(1000, 128, 128, 3)

In [6]:
cats_3.dtype, dogs_3.dtype # look at dtype

(dtype('float64'), dtype('float64'))

In [7]:
dogs_3_y= np.repeat(1,1000) # Let's create training labels for dogs 

In [8]:
cats_3_y = np.repeat(0,1000) # Let's create training labels for cats

In [9]:
# create labels 1=dog, 0=cat
# combine into one array for training labels
y_train_3 = np.concatenate((dogs_3_y, cats_3_y),axis=0)
y_train_3

array([1, 1, 1, ..., 0, 0, 0])

In [10]:
# combine dogs & cats for training 
x_train_3 = np.concatenate((dogs_3,cats_3),axis=0)

In [11]:
# create class names
class_names= ['cats','dogs']

### create train, validate & test set

In [12]:
# create train, validate & test set
from sklearn.model_selection import train_test_split

In [13]:
# initial test train split for 3 colors
x_train_3, x_test_3, y_train_3, y_test_3 = train_test_split(x_train_3, y_train_3, test_size=.30, random_state=55)

In [14]:
# train vs validation set
X_train_3, x_valid_3, y_train_3, y_valid_3 = train_test_split(x_train_3,y_train_3, test_size=.30, random_state=55)

In [15]:
X_train_3.shape

(980, 128, 128, 3)

#### 1st Model

In [16]:
#clear keras session
tf.keras.backend.clear_session()

In [17]:
my_callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=3),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.001)
]
adam_optimizer = keras.optimizers.Adam(learning_rate=0.001,
                                                 beta_1=0.9,
                                                 beta_2=0.999)

sgd_optimizer = keras.optimizers.SGD(lr=.02, momentum=0.9, decay=0.01)

In [18]:
model1 = keras.models.Sequential([
    keras.layers.Conv2D(filters= 32, kernel_size=(3,3),strides=1, activation='relu', padding='same',
                       input_shape=[128,128,3]),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.MaxPooling2D(pool_size=2, strides=2),
    keras.layers.Conv2D(64, (3,3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=2, strides=2),
    keras.layers.Conv2D(124, (3,3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=2, strides=2),
    keras.layers.Flatten(),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(500, activation='relu', kernel_regularizer=keras.regularizers.l2(0.01)),
    keras.layers.Dense(1, activation='sigmoid',kernel_regularizer=keras.regularizers.l2(0.01))
    
    
])

In [19]:
model1.compile(loss='binary_crossentropy', optimizer= sgd_optimizer, metrics=['accuracy'])

# fit model
time_start = datetime.now()
history1 = model1.fit(X_train_3, y_train_3, epochs=15, validation_data=(x_valid_3, y_valid_3), callbacks=my_callbacks)
time_end = datetime.now()
time_elapsed = time_end - time_start

print('Time elapsed')
print(time_elapsed)
print("Fit model on training data")
history1.history

Epoch 1/15
31/31 [==============================] - 49s 2s/step - loss: 10.7933 - accuracy: 0.5296 - val_loss: 12.2402 - val_accuracy: 0.5333
Epoch 2/15
31/31 [==============================] - 50s 2s/step - loss: 8.8992 - accuracy: 0.5980 - val_loss: 8.7059 - val_accuracy: 0.5333
Epoch 3/15
31/31 [==============================] - 49s 2s/step - loss: 7.4316 - accuracy: 0.7541 - val_loss: 8.5728 - val_accuracy: 0.5143
Epoch 4/15
31/31 [==============================] - 46s 1s/step - loss: 6.3700 - accuracy: 0.8612 - val_loss: 7.3208 - val_accuracy: 0.5571
Epoch 5/15
31/31 [==============================] - 46s 1s/step - loss: 5.5734 - accuracy: 0.9469 - val_loss: 6.0608 - val_accuracy: 0.6310
Epoch 6/15
31/31 [==============================] - 45s 1s/step - loss: 5.0207 - accuracy: 0.9469 - val_loss: 5.6679 - val_accuracy: 0.6286
Epoch 7/15
31/31 [==============================] - 45s 1s/step - loss: 4.5499 - accuracy: 0.9878 - val_loss: 5.2937 - val_accuracy: 0.6357
Epoch 8/15
31/31 [

{'loss': [10.793275833129883,
  8.89917278289795,
  7.431638717651367,
  6.3700385093688965,
  5.573376178741455,
  5.02071475982666,
  4.549933433532715,
  4.170104026794434,
  3.873312473297119,
  3.621467113494873,
  3.407027006149292,
  3.212637186050415,
  3.0472865104675293,
  2.899991512298584,
  2.768345832824707],
 'accuracy': [0.5295918583869934,
  0.5979591608047485,
  0.7540816068649292,
  0.8612244725227356,
  0.9469387531280518,
  0.9469387531280518,
  0.9877551198005676,
  0.9989795684814453,
  0.9959183931350708,
  0.9979591965675354,
  1.0,
  1.0,
  1.0,
  1.0,
  0.9979591965675354],
 'val_loss': [12.240228652954102,
  8.705862045288086,
  8.572848320007324,
  7.320805549621582,
  6.060831546783447,
  5.667942047119141,
  5.293745517730713,
  5.027885913848877,
  4.782646656036377,
  4.512895584106445,
  4.27834939956665,
  4.074388027191162,
  3.9095964431762695,
  3.7607033252716064,
  3.6418228149414062],
 'val_accuracy': [0.5333333611488342,
  0.5333333611488342,
 

In [20]:
print("Evaluate on test data")
score = model1.evaluate(x_test_3, y_test_3)
predictions = model1.predict(x_test_3)
predictions

Evaluate on test data
19/19 [==============================] - 7s 361ms/step - loss: 3.5958 - accuracy: 0.6300


array([[7.92374849e-01],
       [1.73886836e-01],
       [3.00467610e-02],
       [2.61945635e-01],
       [7.22060204e-01],
       [5.00066876e-01],
       [4.92382735e-01],
       [6.56456292e-01],
       [7.75033951e-01],
       [9.56792235e-01],
       [9.01351690e-01],
       [6.67703807e-01],
       [7.80081749e-03],
       [9.65804577e-01],
       [1.01504356e-01],
       [3.63904417e-01],
       [8.93299639e-01],
       [6.63640022e-01],
       [2.13618785e-01],
       [7.28161216e-01],
       [1.09116912e-01],
       [1.06752783e-01],
       [4.31978703e-02],
       [9.58617091e-01],
       [7.98100829e-02],
       [9.04900670e-01],
       [3.46979320e-01],
       [5.75347424e-01],
       [1.57440364e-01],
       [7.42434204e-01],
       [9.89000201e-02],
       [9.93900120e-01],
       [2.03935444e-01],
       [4.78095710e-02],
       [8.82412910e-01],
       [5.95434368e-01],
       [2.94900209e-01],
       [3.90898138e-01],
       [8.68434608e-02],
       [2.20206559e-01],


#### 2nd Model

In [21]:
#clear keras session
tf.keras.backend.clear_session()

In [22]:
model2 = keras.models.Sequential([
    keras.layers.Conv2D(filters= 16, kernel_size=3,strides=1, activation='relu', padding='same',
                       input_shape=[128,128,3]),
    keras.layers.BatchNormalization(),
    keras.layers.LeakyReLU(alpha=0.01),
    keras.layers.MaxPooling2D(pool_size=2, strides=2),
    keras.layers.Conv2D(32, (3,3), activation='relu'),
    keras.layers.Conv2D(32, (3,3), activation='relu'),
    keras.layers.Conv2D(32, (3,3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=2, strides=2),
    keras.layers.Conv2D(64, (3,3), activation='relu'),
    keras.layers.Conv2D(64, (3,3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=2, strides=2),
    keras.layers.Conv2D(124, (3,3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=2, strides=2),
    keras.layers.Flatten(),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(525, activation='relu',kernel_regularizer=keras.regularizers.l2(0.02)),
    keras.layers.Dense(1, activation='sigmoid',kernel_regularizer=keras.regularizers.l2(0.02))
    
    
])

In [23]:
model2.compile(loss="binary_crossentropy", optimizer=sgd_optimizer, metrics=["accuracy"])

In [24]:
# fit model
time_start = datetime.now()
history2 = model2.fit(X_train_3, y_train_3, epochs=15, validation_data=(x_valid_3, y_valid_3), callbacks=my_callbacks)
time_end = datetime.now()
time_elapsed = time_end - time_start

print('Time elapsed')
print(time_elapsed)
print("Fit model on training data")
history2.history

Epoch 1/15
31/31 [==============================] - 33s 1s/step - loss: 18.3183 - accuracy: 0.5561 - val_loss: 17.9461 - val_accuracy: 0.5262
Epoch 2/15
31/31 [==============================] - 33s 1s/step - loss: 16.9936 - accuracy: 0.5786 - val_loss: 16.2662 - val_accuracy: 0.5143
Epoch 3/15
31/31 [==============================] - 32s 1s/step - loss: 15.6307 - accuracy: 0.6582 - val_loss: 15.0596 - val_accuracy: 0.5833
Epoch 4/15
31/31 [==============================] - 34s 1s/step - loss: 14.4498 - accuracy: 0.7041 - val_loss: 14.0060 - val_accuracy: 0.5905
Epoch 5/15
31/31 [==============================] - 34s 1s/step - loss: 13.4048 - accuracy: 0.7663 - val_loss: 13.1064 - val_accuracy: 0.5571
Epoch 6/15
31/31 [==============================] - 33s 1s/step - loss: 12.4825 - accuracy: 0.7959 - val_loss: 12.2317 - val_accuracy: 0.6333
Epoch 7/15
31/31 [==============================] - 38s 1s/step - loss: 11.6523 - accuracy: 0.8367 - val_loss: 11.5013 - val_accuracy: 0.6095
Epoch 

{'loss': [18.31830406188965,
  16.99357032775879,
  15.630705833435059,
  14.44980525970459,
  13.404833793640137,
  12.482523918151855,
  11.65231704711914,
  10.914911270141602,
  10.242341041564941,
  9.63267707824707,
  9.095863342285156,
  8.591803550720215,
  8.126599311828613,
  7.711796760559082,
  7.330022811889648],
 'accuracy': [0.5561224222183228,
  0.5785714387893677,
  0.6581632494926453,
  0.704081654548645,
  0.7663265466690063,
  0.795918345451355,
  0.8367347121238708,
  0.8663265109062195,
  0.8979591727256775,
  0.9163265228271484,
  0.9295918345451355,
  0.9459183812141418,
  0.9693877696990967,
  0.9755101799964905,
  0.9744898080825806],
 'val_loss': [17.94614601135254,
  16.26620101928711,
  15.059560775756836,
  14.006031036376953,
  13.106407165527344,
  12.231663703918457,
  11.501269340515137,
  10.810159683227539,
  10.256628036499023,
  9.671541213989258,
  9.18322467803955,
  8.745020866394043,
  8.355172157287598,
  7.969949722290039,
  7.614759922027588

In [25]:
print("Evaluate on test data")
score = model2.evaluate(x_test_3, y_test_3)
predictions = model2.predict(x_test_3)
predictions

Evaluate on test data
19/19 [==============================] - 6s 327ms/step - loss: 7.6401 - accuracy: 0.6600


array([[0.45118907],
       [0.20147914],
       [0.27527654],
       [0.10178003],
       [0.25299746],
       [0.6109521 ],
       [0.19031829],
       [0.6463214 ],
       [0.59359443],
       [0.45272115],
       [0.96552896],
       [0.9687418 ],
       [0.06499529],
       [0.61750585],
       [0.12204486],
       [0.22020632],
       [0.9833946 ],
       [0.54787356],
       [0.2637304 ],
       [0.92602706],
       [0.26937056],
       [0.3211996 ],
       [0.11742172],
       [0.2647354 ],
       [0.10169971],
       [0.30199298],
       [0.11693832],
       [0.17473033],
       [0.28512773],
       [0.473984  ],
       [0.802727  ],
       [0.5277032 ],
       [0.22945768],
       [0.20381698],
       [0.5844843 ],
       [0.96989083],
       [0.7405056 ],
       [0.31322902],
       [0.1712448 ],
       [0.1367681 ],
       [0.27798986],
       [0.57823825],
       [0.52722263],
       [0.54782254],
       [0.7995193 ],
       [0.35271963],
       [0.99951446],
       [0.788

#### 3rd Model

In [26]:
#clear keras session
tf.keras.backend.clear_session()

In [27]:
model3 = keras.models.Sequential([
    keras.layers.Conv2D(filters= 64, kernel_size=3,strides=1, activation='relu', padding='same',
                       input_shape=[128,128,3]),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.MaxPooling2D(pool_size=2),
    keras.layers.Conv2D(128,3, activation='relu',padding='same'),
    keras.layers.Conv2D(128,3, activation='relu',padding='same'),
    keras.layers.MaxPooling2D(pool_size=2),
    keras.layers.Conv2D(256,3, activation='relu',padding='same'),
    keras.layers.Conv2D(256,3, activation='relu',padding='same'),
    keras.layers.MaxPooling2D(pool_size=2),
    keras.layers.Flatten(),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(1000, activation='relu'),
    keras.layers.Dropout(0.25),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dropout(0.25),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(1, activation='sigmoid')
])

In [28]:
model3.compile(loss=tf.keras.losses.binary_crossentropy,
              optimizer=adam_optimizer,
              metrics=['accuracy'])

In [29]:
# fit model
time_start = datetime.now()
history3 = model3.fit(X_train_3, y_train_3, epochs=5, validation_data=(x_valid_3, y_valid_3), callbacks=my_callbacks)
time_end = datetime.now()
time_elapsed = time_end - time_start

print('Time elapsed')
print(time_elapsed)
print("Fit model on training data")
history3.history

Epoch 1/5
31/31 [==============================] - 313s 10s/step - loss: 0.8188 - accuracy: 0.5673 - val_loss: 23.1120 - val_accuracy: 0.4738
Epoch 2/5
31/31 [==============================] - 302s 10s/step - loss: 0.6582 - accuracy: 0.6531 - val_loss: 1.9440 - val_accuracy: 0.5143
Epoch 3/5
31/31 [==============================] - 438s 14s/step - loss: 0.5783 - accuracy: 0.7173 - val_loss: 0.8978 - val_accuracy: 0.6119
Epoch 4/5
31/31 [==============================] - 318s 10s/step - loss: 0.5094 - accuracy: 0.7592 - val_loss: 1.9661 - val_accuracy: 0.5095
Epoch 5/5
31/31 [==============================] - 419s 14s/step - loss: 0.4070 - accuracy: 0.8102 - val_loss: 1.7193 - val_accuracy: 0.4929
Time elapsed
0:30:49.794075
Fit model on training data


{'loss': [0.8188232779502869,
  0.6581867337226868,
  0.5783408284187317,
  0.5094276070594788,
  0.40697863698005676],
 'accuracy': [0.5673469305038452,
  0.6530612111091614,
  0.7173469662666321,
  0.7591836452484131,
  0.8102040886878967],
 'val_loss': [23.111976623535156,
  1.9439804553985596,
  0.8978476524353027,
  1.9660943746566772,
  1.719341516494751],
 'val_accuracy': [0.47380951046943665,
  0.5142857432365417,
  0.6119047403335571,
  0.5095238089561462,
  0.4928571283817291],
 'lr': [0.001, 0.001, 0.001, 0.001, 0.001]}

In [30]:
print("Evaluate on test data")
score = model3.evaluate(x_test_3, y_test_3)
predictions = model3.predict(x_test_3)
predictions

Evaluate on test data
19/19 [==============================] - 44s 2s/step - loss: 1.6444 - accuracy: 0.5300


array([[9.8824155e-01],
       [7.9078948e-01],
       [9.5846832e-01],
       [8.3819085e-01],
       [9.1129863e-01],
       [9.9464905e-01],
       [8.5220838e-01],
       [9.9989295e-01],
       [9.9967992e-01],
       [9.6352321e-01],
       [9.9973679e-01],
       [9.9160147e-01],
       [1.5457302e-02],
       [9.9757814e-01],
       [9.6551502e-01],
       [8.2592320e-01],
       [9.9995416e-01],
       [8.5841662e-01],
       [8.9707851e-01],
       [9.9842513e-01],
       [7.5522721e-01],
       [9.9895501e-01],
       [9.8134381e-01],
       [9.2038167e-01],
       [7.7566028e-01],
       [9.9505997e-01],
       [9.9726456e-01],
       [9.6601474e-01],
       [9.6562451e-01],
       [9.9694705e-01],
       [9.3125945e-01],
       [9.5541763e-01],
       [9.8417962e-01],
       [9.9953610e-01],
       [9.9002618e-01],
       [9.9957931e-01],
       [9.9754030e-01],
       [6.2926590e-01],
       [9.6189547e-01],
       [8.5898840e-01],
       [9.5992064e-01],
       [7.909783

#### 4th Model

In [31]:
#clear keras session
tf.keras.backend.clear_session()

In [32]:
model4 = keras.models.Sequential([
    keras.layers.Conv2D(filters= 64, kernel_size=(3,3),strides=1, activation='relu', padding='same',
                       input_shape=[128,128,3]),
    keras.layers.MaxPooling2D(pool_size=2),
    keras.layers.Conv2D(128,3, activation='relu',padding='same'),
    keras.layers.Conv2D(128,3, activation='relu',padding='same'),
    keras.layers.MaxPooling2D(pool_size=2),
    keras.layers.Conv2D(256,3, activation='relu',padding='same'),
    keras.layers.Conv2D(256,3, activation='relu',padding='same'),
    keras.layers.MaxPooling2D(pool_size=2),
    keras.layers.Flatten(),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(1000, activation='relu'),
    keras.layers.Dropout(0.25),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dropout(0.25),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(1, activation='sigmoid')
])

In [33]:
model4.compile(loss=tf.keras.losses.binary_crossentropy,
              optimizer=adam_optimizer,
              metrics=['accuracy'])

In [34]:
# fit model
time_start = datetime.now()
history4 = model4.fit(X_train_3, y_train_3, epochs=30, validation_data=(x_valid_3, y_valid_3), callbacks= my_callbacks)
time_end = datetime.now()
time_elapsed = time_end - time_start

print('Time elapsed')
print(time_elapsed)
print("Fit model on training data")
history4.history

Epoch 1/30
31/31 [==============================] - 299s 10s/step - loss: 0.8444 - accuracy: 0.5439 - val_loss: 25.7150 - val_accuracy: 0.5333
Epoch 2/30
31/31 [==============================] - 292s 9s/step - loss: 0.7050 - accuracy: 0.5908 - val_loss: 7.0167 - val_accuracy: 0.5262
Epoch 3/30
31/31 [==============================] - 271s 9s/step - loss: 0.6885 - accuracy: 0.6143 - val_loss: 1.4560 - val_accuracy: 0.5381
Epoch 4/30
31/31 [==============================] - 302s 10s/step - loss: 0.6627 - accuracy: 0.6112 - val_loss: 0.9236 - val_accuracy: 0.5167
Epoch 5/30
31/31 [==============================] - 310s 10s/step - loss: 0.6373 - accuracy: 0.6714 - val_loss: 1.2023 - val_accuracy: 0.5429
Epoch 6/30
31/31 [==============================] - 315s 10s/step - loss: 0.6339 - accuracy: 0.6520 - val_loss: 0.9800 - val_accuracy: 0.5143
Epoch 7/30
31/31 [==============================] - 369s 12s/step - loss: 0.6029 - accuracy: 0.6776 - val_loss: 6.8398 - val_accuracy: 0.5262
Time el

{'loss': [0.8443852066993713,
  0.705034613609314,
  0.6884602308273315,
  0.6626622080802917,
  0.6373178362846375,
  0.6339111924171448,
  0.6029068827629089],
 'accuracy': [0.5438775420188904,
  0.5908163189888,
  0.6142857074737549,
  0.6112244725227356,
  0.6714285612106323,
  0.6520408391952515,
  0.6775510311126709],
 'val_loss': [25.715017318725586,
  7.0166778564453125,
  1.456041932106018,
  0.923610270023346,
  1.2023359537124634,
  0.9799841046333313,
  6.839839458465576],
 'val_accuracy': [0.5333333611488342,
  0.526190459728241,
  0.538095235824585,
  0.5166666507720947,
  0.5428571701049805,
  0.5142857432365417,
  0.526190459728241],
 'lr': [0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001]}

In [35]:
print("Evaluate on test data")
score = model4.evaluate(x_test_3, y_test_3)
predictions = model4.predict(x_test_3)
predictions

Evaluate on test data
19/19 [==============================] - 33s 2s/step - loss: 6.8158 - accuracy: 0.4983


array([[1.27226114e-04],
       [8.82110000e-02],
       [1.22290850e-03],
       [2.51635335e-09],
       [4.58403164e-08],
       [4.47760318e-12],
       [2.65091658e-02],
       [1.04125738e-02],
       [2.13006245e-11],
       [3.78674940e-07],
       [3.03776187e-06],
       [1.14622446e-06],
       [1.24368072e-03],
       [1.14142531e-05],
       [2.23059654e-02],
       [5.92576635e-06],
       [2.84119039e-07],
       [2.74881188e-07],
       [1.08402619e-05],
       [8.16530925e-13],
       [6.84475481e-08],
       [3.79763669e-08],
       [3.93901578e-09],
       [3.65524414e-07],
       [8.05856804e-09],
       [2.93638914e-06],
       [1.11454074e-26],
       [2.18816785e-06],
       [1.29887760e-02],
       [1.08780056e-04],
       [6.67846980e-05],
       [9.34910774e-03],
       [1.81111113e-06],
       [1.40855363e-08],
       [1.40442671e-13],
       [2.14064189e-09],
       [4.95433997e-05],
       [1.33159548e-01],
       [5.97819686e-03],
       [1.42276548e-14],


In [41]:
models = [model1, model2, model3, model4]

In [42]:
for i in models:
    df = pd.DataFrame(i.history.history)

In [44]:
for i in models:
    df.plot(figsize=(10,10))
    plt.grid(True)
    plt.gca().set_ylim(0,1)
    plt.title(f"{i._name} graph")
    plt.savefig('computer_vision_graphs')

""
